# Load Required Packages

In [1]:
import pandas as pd
import numpy as np

# Convert to parquet and load the data

In [2]:
# data = pd.read_csv('all_anonymized_2015_11_2017_03.csv',low_memory=False)
# IDXData = data.reset_index()
# IDXData['index'] = 'AD-' + IDXData['index'].astype('str')
# Summary = IDXData.describe(include='all')
# IDXData.to_parquet('ParquetData.parquet')

In [3]:
Data = pd.read_parquet("ParquetData.parquet")
Data.head(2)

,index,maker,model,mileage,manufacture_year,engine_displacement,engine_power,body_type,color_slug,stk_year,transmission,door_count,seat_count,fuel_type,date_created,date_last_seen,price_eur
0,AD-0,ford,galaxy,151000.0,2011.0,2000.0,103.0,None,None,NaN,man,5.0,7.0,diesel,2015-11-14 18:10:06.838319+00,2016-01-27 20:40:15.46361+00,10584.75
1,AD-1,skoda,octavia,143476.0,2012.0,2000.0,81.0,None,None,NaN,man,5.0,5.0,diesel,2015-11-14 18:10:06.853411+00,2016-01-27 20:40:15.46361+00,8882.31


In [4]:
Data = Data[(Data['maker'].notnull())&(Data['model'].notnull())]

In [5]:
MostImportantColumnsData = Data[
    [
        "index",
        "maker",
        "model",
        "mileage",
        "manufacture_year",
        "engine_displacement",
        "engine_power",
        "body_type",
        "stk_year",
        "transmission",
        "door_count",
        "seat_count",
        "fuel_type",
        "price_eur",
    ]
].copy()

In [6]:
ReasonAblePrice = MostImportantColumnsData[
    (MostImportantColumnsData["price_eur"] >= 500)
    & (MostImportantColumnsData["price_eur"] <= 50000)
    & (MostImportantColumnsData["price_eur"] != 1295.34)
].copy()
len(ReasonAblePrice)

1779127

In [7]:
ReasonAblePrice["Nulls"] = ReasonAblePrice.isnull().sum(axis=1)

print(len(ReasonAblePrice.columns))
print(len(ReasonAblePrice))

ReasonAblePrice.groupby("Nulls")["index"].count()

15
1779127


Nulls
0      5024
1    229749
2    879909
3    245652
4    170815
5     85705
6     70017
7     83572
8      7050
9      1634
Name: index, dtype: int64

In [8]:
LowNullData = ReasonAblePrice.fillna(value=np.nan)
LowNullData = LowNullData.dropna(thresh=6)

len(LowNullData)

1779127

In [9]:
LowNullData.head()

,index,maker,model,mileage,manufacture_year,engine_displacement,engine_power,body_type,stk_year,transmission,door_count,seat_count,fuel_type,price_eur,Nulls
0,AD-0,ford,galaxy,151000.0,2011.0,2000.0,103.0,NaN,NaN,man,5.0,7.0,diesel,10584.75,2
1,AD-1,skoda,octavia,143476.0,2012.0,2000.0,81.0,NaN,NaN,man,5.0,5.0,diesel,8882.31,2
3,AD-3,skoda,fabia,111970.0,2004.0,1200.0,47.0,NaN,NaN,man,5.0,5.0,gasoline,2960.77,2
4,AD-4,skoda,fabia,128886.0,2004.0,1200.0,47.0,NaN,NaN,man,5.0,5.0,gasoline,2738.71,2
5,AD-5,skoda,fabia,140932.0,2003.0,1200.0,40.0,NaN,NaN,man,5.0,5.0,gasoline,1628.42,2


# Clean engine_displacement

In [10]:
EDP_Groups = pd.DataFrame(
    LowNullData.groupby(
        ["maker", "model", "manufacture_year", "transmission", "body_type"]
    )["engine_displacement"].median()
)
EDP_Groups = EDP_Groups[EDP_Groups["engine_displacement"].notnull()]

In [11]:
CleanedEngineDisplacement = LowNullData.copy()
CleanedEngineDisplacement["engine_displacement"] = CleanedEngineDisplacement[
    "engine_displacement"
].apply(lambda x: x if x > 600 and x < 8500 else np.nan)

In [12]:
def CleanEngineDisplacement(s: pd.Series):
    if np.isnan(s["engine_displacement"]):
        idx = (
            s["maker"],
            s["model"],
            s["manufacture_year"],
            s["transmission"],
            s["body_type"],
        )
        if idx in EDP_Groups.index:
            return EDP_Groups.loc[idx].median()
        idx = (
            s["maker"],
            s["model"],
            s["manufacture_year"],
            s["transmission"],
        )
        if idx in EDP_Groups.index:
            return EDP_Groups.loc[idx].median()
        idx = (
            s["maker"],
            s["model"],
            s["manufacture_year"],
        )
        if idx in EDP_Groups.index:
            return EDP_Groups.loc[idx].median()
        idx = (
            s["maker"],
            s["model"],
        )
        if idx in EDP_Groups.index:
            return EDP_Groups.loc[idx].median()
    return s["engine_displacement"]


CleanedEngineDisplacement["engine_displacement"] = CleanedEngineDisplacement.apply(
    lambda x: CleanEngineDisplacement(x), axis=1
)

In [16]:
len(CleanedEngineDisplacement[CleanedEngineDisplacement["engine_displacement"].isnull()])

51

In [14]:
# ReasonAbleData["engine_displacement"].isna().sum()

In [15]:
# Hassan Columns
# engine_displacement, engine_power, body_type, stk_year